# Designers Image Classification 


## Load external libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.contrib.layers import flatten
from keras.utils.np_utils import to_categorical
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers import Conv2D
from keras.layers.pooling import MaxPooling2D
import cv2
import glob

/Users/hhan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/hhan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Designers Classification

**Steps**:
1. Data preparation:
    - Load the images and labels
    - Split the data into two sets:  training set and testing set
2. Build a Convolutional Neural Network model (CNN). 
3. Train the model with the training data and test the accuracy using testing set.

### Step 1:  Load the images and labels

In [13]:
import os
from hotdog import load_images, shuffle_samples, load_image_with_label
import pandas as pd 

image_size = (114, 114)

# for each image, we will produce one rotated image along with the original image
n_rotated = 1
# number of class, 1 is hotdog, 0 is not hotdog
n_classes = 2

# get all the hotdog and not hotdog images paths
designers = [i for i in os.listdir("/Users/hhan/fashion/designer_image") if not i.startswith(".")]
image_dir = "/Users/hhan/fashion/designer_image"

images = []
labels = []

for i in range(len(designers)):
    print (designers[i])
    designer_dir = os.path.join(image_dir, designers[i])
    designer_images = [os.path.join(designer_dir, i) for i in os.listdir(designer_dir) if not i.startswith(".")]
    
    hotdog_images, hotdog_labels = load_image_with_label(designer_images, i, image_size, n_rotated)
    images.append(hotdog_images)
    labels.append(hotdog_labels)
    
all_images = np.concatenate(images, axis=0)
all_labels = np.concatenate(labels, axis=0)
    
    

vera wang
marc by marc jacobs
carolina herrera
calvin klein
blumarine
missoni


In [14]:
# images are all the images combining hotdog and not hotdog images
print (all_images.shape)
print (all_labels.shape)

(10290, 114, 114, 3)
(10290,)


#### Split the dataset  into training and testing set

In [15]:
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.1, random_state=rand_state)
print("number of training samples", np.array(X_train).shape[0])
print("number of testing samples", np.array(X_test).shape[0])

number of training samples 9261
number of testing samples 1029


###  Step 2 : Build the model

In [16]:
def kerasModel(inputShape):
    # feature learning 
    model = Sequential()
    
    # convolutional block 1 
    model.add(Convolution2D(32, (5, 5), input_shape=inputShape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # convolutional block 2
    model.add(Convolution2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
   
    # convolutional block 3
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # classification
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile('adam', 'binary_crossentropy', ['accuracy'])
    return model

###  Step 3 : Train and evalutate


In [17]:
inputShape = (114, 114, 3)
model = kerasModel(inputShape)
history = model.fit(X_train, y_train, nb_epoch=10, validation_split=0.1)

metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))
    


/Users/hhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 8334 samples, validate on 927 samples
Epoch 1/10
8334/8334 [==============================] - 99s 12ms/step - loss: -19.6406 - acc: 0.1955 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 2/10
8334/8334 [==============================] - 96s 12ms/step - loss: -19.8849 - acc: 0.1958 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 3/10
8334/8334 [==============================] - 113s 14ms/step - loss: -19.8849 - acc: 0.1958 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 4/10
8334/8334 [==============================] - 114s 14ms/step - loss: -19.8849 - acc: 0.1958 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 5/10
8334/8334 [==============================] - 109s 13ms/step - loss: -19.8849 - acc: 0.1958 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 6/10
8334/8334 [==============================] - 112s 13ms/step - loss: -19.8849 - acc: 0.1958 - val_loss: -20.6890 - val_acc: 0.1899
Epoch 7/10
8334/8334 [==============================] - 111s 13ms/step - loss: -19.8849 - acc: 0.1958 - val_l

# Save the model 

In [56]:
model.save('model.h5') 